In [4]:
import os

machine_path = os.getcwd()
user_root_dir = '/'.join(machine_path.split('/')[:-2])

import torch
import sys

# from torchvision import datasets, models, transforms

from PIL import Image
import torchvision

sys.path.append('%s/training_scaffold_own/res/'%user_root_dir)
from models.models import get_model
from loader.loader import get_loader
import random
import pickle

# Learning rate parameters
BASE_LR = 0.001
EPOCH_DECAY = 30 # number of epochs after which the Learning rate is decayed exponentially.
DECAY_WEIGHT = 0.1 # factor by which the learning rate is reduced.

# DATASET INFO
NUM_CLASSES = 2 # set the number of classes in your dataset
DATA_DIR = '../data/' # to run with the sample dataset, just set to 'hymenoptera_data'

# DATALOADER PROPERTIES
BATCH_SIZE = 10 # Set as high as possible. If you keep it too high, you'll get an out of memory error.


### GPU SETTINGS
CUDA_DEVICE = 0 # Enter device ID of your gpu if you want to run on gpu. Otherwise neglect.
GPU_MODE = 1 # set to 1 if want to run on gpu.

# MODEL_ARCH = 'transformer'

from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import copy
import os
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import collections
from collections import OrderedDict
from tqdm import tqdm

In [14]:
DATASET_NAME = 'image_test_v7_cma_resnet18_antialiased_v7_40_normalized_final'
model_path = '../training_models/saved_models/resnet18_antialiased_v7_40_normalized_final.pt'

# DATASET_NAME = 'image_test_v7_cma_resnet18_v7'
# model_path = '../training_models/saved_models/resnet18_v7_normalized.pt'

# DATASET_NAME = 'image_test_v7_cma_resnet18_v7_40'
# model_path = '../training_models/saved_models/resnet18_v7_40_final.pt'

# DATASET_NAME = 'image_test_v7_cma_resnet18_v7_anti_aliased'
# model_path = '../training_models/saved_models/resnet18_antialiased_v7_normalized_final.pt'

# DATASET_NAME = 'image_test_v7_cma_resnet18_v7_truly_shift_invariant'
# model_path = '../training_models/saved_models/resnet18_v7_truly_shift_invariant_normalized_final.pt'

In [15]:
# DATASET_NAME = 'image_test_v7_cma_resnet18_v7_subsampled_unseen'
# model_path = '../training_models/saved_models/resnet18_v7_subsampled_normalized_final.pt'

# DATASET_NAME = 'image_test_v7_cma_resnet18_v7_unseen'
# model_path = '../training_models/saved_models/resnet18_v7_normalized.pt'

# DATASET_NAME = 'image_test_v7_cma_resnet18_v7_40_unseen'
# model_path = '../training_models/saved_models/resnet18_v7_40_final.pt'

# DATASET_NAME = 'image_test_v7_cma_resnet18_v7_anti_aliased_unseen'
# model_path = '../training_models/saved_models/resnet18_antialiased_v7_normalized_final.pt'

# DATASET_NAME = 'image_test_v7_cma_resnet18_v7_truly_shift_invariant_unseen'
# model_path = '../training_models/saved_models/resnet18_v7_truly_shift_invariant_normalized_final.pt'

In [16]:
NUM_EPOCHS = 2
BATCH_SIZE = 25


image_transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
])

GPU = 1

NUM_CLASSES = 55

In [17]:
loader_new = get_loader('multi_attribute_loader_file_list_shapenet')
if 'smadan' in user_root_dir:
    file_list_root = '%s/dataset_lists_openmind'%user_root_dir
elif 'spandan' in user_root_dir:
    file_list_root = "%s/dataset_lists_fasrc"%user_root_dir


att_path = '%s/differentiable_graphics_ml/training_models/shapenet_id_to_class_num.p'%user_root_dir
shuffles = {'train':True,'val':True,'test':True,'seen_test':False}

In [18]:
################ GET FROM USER CONFIG - TODO #####################
file_lists = {}
dsets = {}
dset_loaders = {}
dset_sizes = {}
for phase in ['test']:
    file_lists[phase] = "%s/%s_list_%s.txt"%(file_list_root,phase,DATASET_NAME)
    dsets[phase] = loader_new(file_lists[phase],att_path, image_transform)
    dset_loaders[phase] = torch.utils.data.DataLoader(dsets[phase], batch_size=BATCH_SIZE, shuffle = shuffles[phase], num_workers=0,drop_last=True)
    dset_sizes[phase] = len(dsets[phase])

In [19]:
# os.listdir('../training_models/saved_models/')

In [20]:
# model_path = '../training_models/saved_models/resnet18_train_v3_fixed_lighting.pt'
# model_path = '../training_models/saved_models/resnet18_v7_normalized.pt'
# model_path = '../training_models/saved_models/resnet18_v3_symmetric_bias_6_conditions_camera.pt'
# model_path = '../training_models/saved_models/resnet18_v7_normalized.pt'
# model_path = '../training_models/saved_models/resnet18_v7_fine_tuned_less_frozen.pt'
# model_path = '../training_models/saved_models/resnet18_v7_subsampled_normalized_final.pt'
# model_path = '../training_models/saved_models/resnet18_v7_truly_shift_invariant_normalized_final.pt'


# model_path = '../training_models/saved_models/resnet18_v6_normalized_final.pt'
loaded_model = torch.load(model_path)
loaded_model.eval();
loaded_model.cuda();

In [21]:
def input_shift_batch(arr, w_shift, h_shift):
    empty_arr = torch.zeros((arr.shape))
    
    if w_shift <0 and h_shift<0:
        empty_arr[:,:,:224+h_shift,:224+w_shift] = arr[:,:,-h_shift:,-w_shift:]
    elif w_shift >=0 and h_shift >=0:
        empty_arr[:,:,h_shift:,w_shift:] = arr[:,:,:arr.shape[2]-h_shift,:arr.shape[3]-w_shift]
    elif w_shift >=0 and h_shift <0:
        empty_arr[:,:,:224+h_shift,w_shift:] = arr[:,:,-h_shift:,:arr.shape[3]-w_shift]
    elif w_shift <0 and h_shift >=0:
        empty_arr[:,:,h_shift:,:224+w_shift] = arr[:,:,:arr.shape[2]-h_shift,-w_shift:]
    
    return empty_arr

25

In [53]:
all_errors = 0
total = 0
for phase in ['test']:
    for data in tqdm(dset_loaders[phase]):
        count = 0
        batch_predictions = torch.zeros((16, BATCH_SIZE))
        for w_shift in range(-2, 2):
            for h_shift in range(-2, 2):
                if count % 10 == 0:
                    print("%s"%count)
                inputs_orig, labels, image_paths = data
                inputs_shifted = input_shift_batch(inputs_orig, w_shift, h_shift)
                inputs = inputs_shifted.float().cuda()
                labels = labels.long().cuda()
                
                im_means = torch.mean(inputs.view(BATCH_SIZE, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
                im_stds = torch.std(inputs.view(BATCH_SIZE, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
                inputs = (inputs - im_means)/im_stds

                outputs = loaded_model(inputs)
                preds = torch.argmax(outputs,dim=1)
                batch_predictions[count] = preds
                count += 1
                batch_repeated_labels = labels.repeat(16, 1)
        shifted_matches = batch_predictions.cpu() == batch_repeated_labels.cpu()
        batch_errors = torch.sum(torch.sum(shifted_matches,dim=0) != len(shifted_matches))
        all_errors += batch_errors
        total += data[0].shape[0]

  0%|          | 0/3 [00:00<?, ?it/s]

0/100
10/100


 33%|███▎      | 1/3 [00:01<00:03,  1.68s/it]

0/100
10/100


 67%|██████▋   | 2/3 [00:03<00:01,  1.57s/it]

0/100
10/100


100%|██████████| 3/3 [00:04<00:00,  1.53s/it]


In [54]:
1 - all_errors/total

tensor(0.9200)

In [32]:
# batch_repeated_labels[0]

In [31]:
# batch_predictions[1]

In [173]:
all_matches_stacked= torch.hstack(all_matches)
shift_attack_accuracy = torch.sum(torch.sum(all_matches_stacked, dim=0) == 100)/100
print(shift_attack_accuracy)

tensor(0.6800)


# Checking for consistency

In [33]:
all_preds = torch.zeros((100, BATCH_SIZE))
count = 0
for w_shift in range(-5,5):
    for h_shift in range(-5,5):
        if count % 10 == 0:
            print("%s/100"%count)
        inputs_orig, labels, image_paths = data
        inputs_shifted = input_shift_batch(inputs_orig, w_shift, h_shift)
        inputs = inputs_shifted.float().cuda()
        labels = labels.long().cuda()
        if 'normalize' in model_path:
            im_means = torch.mean(inputs.view(BATCH_SIZE, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
            im_stds = torch.std(inputs.view(BATCH_SIZE, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
            inputs = (inputs - im_means)/im_stds
            
        outputs = loaded_model(inputs)
        preds = torch.argmax(outputs,dim=1)
        all_preds[count] = preds        
        count += 1

0/100


TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [30]:
for i in range(25):
    print(torch.sum(all_corrects[i,:]).item()/25)

0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88
0.88


In [16]:
# all_corrects = 0
# total = 0
# incorrect_predicted_images = []
# all_paths = []
# for phase in ['test']:
#     for data in tqdm(dset_loaders[phase]):
#         inputs, labels, image_paths = data
#         inputs = inputs.float().cuda()
#         labels = labels.long().cuda()
#         if 'normalize' in model_path:
#             im_means = torch.mean(inputs.view(BATCH_SIZE, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
#             im_stds = torch.std(inputs.view(BATCH_SIZE, -1),dim=1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
#             inputs = (inputs - im_means)/im_stds

#         outputs = loaded_model(inputs)
#         preds = torch.argmax(outputs,dim=1)
#         corrects = torch.sum(preds == labels).item()
#         all_corrects += corrects
#         total += len(preds)
#         all_paths.extend(image_paths)
#         incorrect_predicted_images.extend([image_paths[i] for i in torch.where(preds!=labels)[0]])

In [60]:
print(all_corrects/total)

0.6301522842639594


In [61]:
acc = all_corrects/total

In [62]:
print('10 3D models seen: %s'%acc)

10 3D models seen: 0.6301522842639594


In [43]:
print('4 3D models seen: %s'%acc)

4 3D models seen: 0.5473096446700507


In [64]:
model_predictions = {}
model_predictions['correct'] = [i for i in all_paths if i not in incorrect_predicted_images]
model_predictions['incorrect'] = incorrect_predicted_images

In [65]:
len(model_predictions['correct'])

4599

In [66]:
len(model_predictions['incorrect'])

5451

In [67]:
len(model_predictions['correct'])/len(all_paths)

0.45761194029850744

In [74]:
t.shape

torch.Size([3, 224, 224])

In [68]:
'/om5/user/smadan/differentiable_graphics_ml/data/train_v7_shapenet/%s_v7_test.p'%model_name

'/om5/user/smadan/differentiable_graphics_ml/data/train_v7_shapenet/resnet18_v7_fine_tuned_less_frozen_v7_test.p'

In [69]:
with open('/om5/user/smadan/differentiable_graphics_ml/data/train_v7_shapenet/%s_v7_test.p'%model_name, 'wb') as F:
    pickle.dump(model_predictions, F)

In [66]:
# with open('/om5/user/smadan/differentiable_graphics_ml/data/train_v7_shapenet/resnet18_incorrect_predictions.p', 'wb') as F:
#     pickle.dump(incorrect_predicted_images, F)    

In [42]:
acts.shape

torch.Size([25, 512, 7, 7])